### Projeto Consulta Jurídica

In [84]:
# imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.alert import Alert 
import pandas as pd
import time
import os

servico = Service(ChromeDriverManager().install()) # mantém o chromedriver atualizado 
navegador = webdriver.Chrome(service=servico)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Suporte\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [85]:
# identifica caminho para o index
caminho = os.getcwd()
arquivo = caminho + r"\index.html"
navegador.get(arquivo)

In [86]:
def trata_alertas():
    # tratar alertas
    on_alert = True
    while on_alert: 
        time.sleep(5)
        alerta = Alert(navegador)
        if alerta.text == "Vamos buscar no nosso banco de dados, por favor aguarde. Pode durar alguns minutos.":
            alerta.accept()
        elif alerta.text == "Nenhum processo encontrado!":
            alerta.accept()
            status = "Nenhum processo encontrado."
            print('Nenhum processo encontrado')
            on_alert = False
        elif alerta.text == "Processo encontrado com sucesso! Deseja acessar?":
            alerta.accept()
            status = 'Processo encontrado.'
            print('Processo encontrado')
            on_alert = False
    return status

In [90]:
# import base de dados
df = pd.read_excel('Processos.xlsx')
display(df)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [88]:
# preencher campos 
for linha in df.index:
    
    aba_original = navegador.window_handles[0]
    navegador.switch_to.window(aba_original)
    cidade = df.loc[linha, 'Cidade']
    
    # get element
    elemento = navegador.find_element(By.XPATH, '/html/body/div/div/button')
    
    # create action chain object and move to element
    ActionChains(navegador).move_to_element(elemento).perform()
    cidade = navegador.find_element(By.PARTIAL_LINK_TEXT, cidade)
    cidade.click()

    # identificar e mudar para nova aba
    nova_aba = navegador.window_handles[-1]
    navegador.switch_to.window(nova_aba)
    
    # nome
    nome = df.loc[linha, 'Nome']
    navegador.find_element(By.XPATH, '//*[@id="nome"]').send_keys(nome)

    # advogado 
    advogado = df.loc[linha, 'Advogado']
    navegador.find_element(By.XPATH, '//*[@id="advogado"]').send_keys(advogado)

    # número do processo 
    processo = df.loc[linha, 'Processo']
    navegador.find_element(By.XPATH, '//*[@id="numero"]').send_keys(processo)

    # apertar botão pesquisar
    navegador.find_element(By.XPATH, '//*[@id="formulario"]/div/button').click()
    
    # chama função que trata alertas
    status = trata_alertas()
    df.loc[linha, 'Status'] = status
    navegador.close()
    
navegador.quit()

Nenhum processo encontrado
Nenhum processo encontrado
Processo encontrado
Nenhum processo encontrado


In [89]:
display(df)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Nenhum processo encontrado.
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Nenhum processo encontrado.
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Processo encontrado.
3,Carol,Amanda,PC5197,São Paulo,Nenhum processo encontrado.
